<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/>
        Importando Bibliotecas
    </strong>
</p>

In [2]:
import pandas as pd
import xlwings as xw

<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/>
        Lendo base de dados
    </strong>
</p>

In [4]:
def ler_arquivo(nome_arquivo):
    try:
        return pd.read_excel(f'bases/{nome_arquivo}.xlsx')
    except FileNotFoundError as e:
        return f'Erro: {e}'

In [6]:
df = ler_arquivo('dados de entregas')

<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/>
        SLA
    </strong>
</p>

In [ ]:
# # Jeito 1 - Filtrando dentro do loc

# # Total de entregas sem atraso
# # sla_ok = df.loc[ (df['dias_atraso'] == 0) & 
# #     (df['status'] == 'ENTREGUE')].shape[0] 
# # sla_ok

# # # Total de entregas
# # df_total_entregas = df.loc[ df['status'] == 'ENTREGUE'].shape[0]
# # df_total_entregas

# # # percentual SLA
# # sla_percentual = (sla_ok / df_total_entregas) * 100

# print(f'{round(sla_percentual,2)}% das entregas foram realizadas dentro do prazo!')

37.24% das entregas foram realizadas dentro do prazo!


In [ ]:
# Jeito 2 - usando a tabela criada para total entregas

# Total de entregas
df_total_entregas = df.loc[ df['status'] == 'ENTREGUE']
df_total_entregas

# Total de entregas sem atraso
sla_ok = df_total_entregas.loc[(df_total_entregas['dias_atraso'] == 0)]
sla_ok

# percentual SLA
sla_percentual = (sla_ok.shape[0] / df_total_entregas.shape[0]) * 100

print(f'{round(sla_percentual,2)}% das entregas foram realizadas dentro do prazo!')

37.24% das entregas foram realizadas dentro do prazo!


<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/>
        KPI - Entregas Atrasadas
    </strong>
</p>

In [ ]:
# Resultado utilizando o jeito 1

# entregas_atrasadas = df.loc[ 
    # (df['dias_atraso'] > 0) & 
    # (df['status'] == 'ENTREGUE')].shape[0]
# entregas_atrasadas

# sla_percentual_atrasadas = (entregas_atrasadas/df_total_entregas) * 100
# sla_percentual_atrasadas

# print(f'{round(sla_percentual_atrasadas,2)}% das entregas foram realizadas fora do prazo!')

62.76% das entregas foram realizadas fora do prazo!


In [ ]:
# Resultado utilizando jeito 2

percentual_atraso = (
    df_total_entregas.loc [ df_total_entregas['dias_atraso'] > 0].shape[0] / 
    df_total_entregas.shape[0]
) * 100

round(percentual_atraso,2)

62.76

<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/>
        KPI - Análise por Filial
    </strong>
</p>

In [ ]:
# Média de dias de atraso por filial
sla_filial = df_total_entregas.groupby('filial')['dias_atraso'].mean()

round(sla_filial,2)

filial
MG    1.84
PR    1.92
RJ    1.81
RS    1.97
SC    1.91
SP    1.80
Name: dias_atraso, dtype: float64

In [ ]:

# lambda cria a função sem precisar nomea-la

,filial,% SLA no Prazo
0,RJ,42.08
1,PR,38.34
2,MG,38.25
3,SP,35.76
4,RS,34.93
5,SC,34.76
